In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

# General libraries
import os
import numpy as np
import pandas as pd 
import random
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

# Deep learning libraries
import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, SeparableConv2D, MaxPooling2D, LeakyReLU, Activation, Lambda, GlobalAveragePooling2D, DepthwiseConv2D, GlobalMaxPooling2D, Conv2DTranspose, GaussianNoise
from keras.layers import Add, Concatenate
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.optimizers import Adam, SGD
from keras import regularizers
import tensorflow as tf


# Setting seeds for reproducibility
seed = 232
np.random.seed(seed)
tf.random.set_seed(seed)

In [ ]:
img_dims = 227
epochs = 50
batch_size = 8

In [ ]:
input_path = '/content/drive/My Drive/Cross Validation Data/Fold 1/'

In [ ]:
X_train = []
X_test = []
y_train = []
y_test = []

for tt in ['train', 'test']:
  for cond in ['/COVID19/', '/NORMAL/','/PNEUMONIA/']:
    for img_name in os.listdir(input_path+tt+cond):
      img = cv2.imread(input_path+tt+cond+img_name, 0)
      try:
        img = cv2.resize(img, (img_dims, img_dims))
      except:
        print(img_name)
      
      #lbp_img = local_binary_pattern(img, P=8, R=8, method='uniform')/255.0
      img = np.dstack([img, img, img])  #Feinting color image channel
      img = img.astype('float32') / 255.0
      #img = img/255.0
      if tt=='train':
        X_train.append(img)
        
        if cond=='/COVID19/':
          label=0
          y_train.append(label)
        elif cond=='/NORMAL/':
          label=1
          y_train.append(label)
        else:
          label=2
          y_train.append(label)
      else:
        X_test.append(img)
        
        if cond=='/COVID19/':
          label=0
          y_test.append(label)
        elif cond=='/NORMAL/':
          label=1
          y_test.append(label)
        else:
          label=2
          y_test.append(label)


X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
print(X_train.shape, X_test.shape)

In [ ]:
from keras.utils import to_categorical
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

print(y_train_cat.shape, y_test_cat.shape)

In [ ]:
from keras.applications import MobileNet, DenseNet201, NASNetMobile,InceptionV3

def create_cnn(img_dims):
  model = DenseNet201(input_shape=(img_dims, img_dims, 3), weights=None, include_top=False,pooling='avg')

  #x = GlobalAveragePooling2D(name='GlobalAvgPool')(model.output)
  x = Dense(256, activation='relu', name='Bottleneck1')(model.output)
  x = Dense(128, activation='relu', name='Bottleneck2')(x)
  x=Dropout(.2)(x)
  x = Dense(2, activation='softmax')(x)

  model_gen = Model(model.input, x)

  return model_gen


In [ ]:
from keras.applications import MobileNet, DenseNet201, NASNetMobile,InceptionV3

def create_cnn1(img_dims):
  model = DenseNet201(input_shape=(img_dims, img_dims, 3), weights='imagenet', include_top=False,pooling='avg')
  
  for layers in model.layers[:]:
     layers.trainable = True

  #x = GlobalAveragePooling2D(name='GlobalAvgPool')(model.output)
  x = Dense(256, activation='relu', name='Bottleneck1')(model.output)
  x = Dense(128, activation='relu', name='Bottleneck2')(x)
  x=Dropout(.2)(x)
  x = Dense(2, activation='softmax')(x)

  model_gen = Model(model.input, x)
  
  

  return model_gen


In [ ]:
normpneu_model_name = 'normpneu_FOLD1' 
normbact_model_name = 'normbact_FOLD1' #You have to change Fold Name Here...................
normvir_model_name = 'normvir_FOLD1'  #You have to change Fold Name Here...................
  

normpneu_model = create_cnn(img_dims)
normbact_model = create_cnn(img_dims)
normvir_model = create_cnn(img_dims)


for layer in normpneu_model.layers:
  layer._name = layer._name + 'normpneu'

for layer in normbact_model.layers:
  layer._name = layer._name + 'normbact'

for layer in normvir_model.layers:
  layer._name = layer._name + 'normvir'




opt = Adam(lr=0.001)
#....................We need to tune weights here.......................#
normbact_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
weight_path1 = '/content/drive/My Drive/Weights/2class_DenseNet201_normbact_dataaug_new.h5'#...................Change it As necessary....................#'################################
normbact_model.load_weights(weight_path1)



normpneu_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
weight_path2 = '/content/drive/My Drive/Weights/2class_DenseNet201_normpneu_dataaug_new.h5'#...................Change it As necessary....................#'################################
normpneu_model.load_weights(weight_path2)

normvir_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
weight_path3 = '/content/drive/My Drive/Weights/2class_DenseNet201_normvir_dataaug_new.h5'#...................Change it As necessary....................#'################################
normvir_model.load_weights(weight_path3)



In [ ]:
trainable_model_name = 'trainable_FOLD1'  #You have to change Fold Name Here...................

trainable_model = create_cnn1(img_dims)

for layer in trainable_model.layers:
  layer._name = layer._name + 'trainable'



opt = Adam(lr=0.001)
#....................We need to tune weights here.......................#
trainable_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])




trainable_model.summary()

# FOR ENSEMBLE MODEL

In [ ]:
normpneu_feat = Model(normpneu_model.input, normpneu_model.layers[-5].output)
#x1=(Dense(128,activation='relu'))(normpneu_feat.output)
#x1=(Dense(128,activation='relu'))(x1)
model1 = Model(normpneu_feat.input,normpneu_feat.output )

normbact_feat = Model(normbact_model.input, normbact_model.layers[-5].output)
#x2=(Dense(256,activation='relu'))(normbact_feat.output)
#x2=(Dense(128,activation='relu'))(x2)

model2 = Model(normbact_feat.input, normbact_feat.output)

normvir_feat = Model(normvir_model.input, normvir_model.layers[-5].output)
#x3=(Dense(256,activation='relu'))(normvir_feat.output)
#x3=(Dense(128,activation='relu'))(x3)
model3 = Model(normvir_feat.input, normvir_feat.output)



trainable_feat = Model(trainable_model.input, trainable_model.layers[-5].output)
#model_first = Model(model.inputs, model.layers[-3].output)

# First we will try freezing the layers and then feature concatenating. Later we can try training the whole network
for layer in model1.layers[:]:
  layer.trainable = True

for layer in model2.layers[:]:
  layer.trainable = True

for layer in model3.layers[:]:
  layer.trainable = True

#for layer in trainable_feat.layers[:]:
 # layer.trainable = False  

for layer in trainable_feat.layers[:]:
  layer.trainable = True  

x = Concatenate()([trainable_feat.output,model1.output,model2.output,model3.output])
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x =  Dropout(.2)(x)
x = Dense(3, activation='softmax')(x)

model = Model([trainable_feat.input,model1.input,model2.input,model3.input], x)

#....................We need to tune weights here.......................#
opt=Adam(lr=.0001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
######## THIS IS FOR COMBINED MODEL ########
weight_save_path = '/content/drive/My Drive/Results/Weight/'
model_name = 'DenseNet201_Ensemble_scheme_1_fold_1_FT'   #You have to change Fold Name Here...................

lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=2, verbose=2, mode='max', min_lr=0.000001)

checkpoint = ModelCheckpoint(weight_save_path + model_name+ '.h5', monitor='val_accuracy', save_best_only=True, save_weights_only=True)
hist = model.fit([X_train,X_train,X_train,X_train], y_train_cat,epochs=epochs, batch_size=8,  validation_split=0.2, callbacks=[checkpoint,lr_reduce])


In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 3))
ax = ax.ravel()

for i, met in enumerate(['accuracy', 'loss']):
    ax[i].plot(hist.history[met])
    ax[i].plot(hist.history['val_' + met])
    ax[i].set_title('Model {}'.format(met))
    ax[i].set_xlabel('epochs')
    ax[i].set_ylabel(met)
    ax[i].legend(['train', 'val'])


plt.savefig('/content/drive/My Drive/Results/Graphs/'+model_name+'.png',dpi=500)

In [ ]:
model.save_weights(weight_save_path + model_name + '.h5')

In [ ]:
from sklearn.metrics import classification_report
from tensorflow.keras.utils import plot_model

y_pred = model.predict([X_test,X_test,X_test,X_test])

y_pred_bool = np.argmax(y_pred, axis=1)
print(np.unique(y_pred_bool))
report = classification_report(y_test, y_pred_bool, output_dict=True)
print(classification_report(y_test, y_pred_bool))


############## Make Sure to Change this ####################
plot_save_path = '/content/drive/My Drive/Results/ModelPlot/'
hist_save_path = '/content/drive/My Drive/Results/History/'
result_save_path = '/content/drive/My Drive/Results/Result/'
confusion_matrix_save_path = '/content/drive/My Drive/Results/Confusion Matrix/'


hist_df = pd.DataFrame(hist.history)
hist_csv_file = hist_save_path+model_name+'_history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

plot_model(model, plot_save_path + model_name+'.png', show_shapes=True)

result_df = report
result_df = pd.DataFrame(result_df).transpose()

print(result_df)

result_df.to_csv(result_save_path + model_name + '_result.csv')

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
from seaborn import heatmap
from matplotlib import pyplot as plt

preds = np.argmax(model.predict([X_test,X_test,X_test,X_test]), axis=1)

acc = accuracy_score(y_test, np.round(preds))*100
cm = confusion_matrix(y_test, np.round(preds))
cm_norm = confusion_matrix(y_test, np.round(preds), normalize='true')

print('CONFUSION MATRIX ------------------')

ax = heatmap(cm, cmap='Blues', linecolor='lightblue',linewidths=.5,annot=True,annot_kws={'size': 14,"weight": "bold"}, xticklabels=['COVID19', 'NORMAL','PNEUMONIA'], yticklabels=['COVID19', 'NORMAL','PNEUMONIA'], square=True, fmt='d')

ax.set_xticklabels(ax.get_xmajorticklabels(), fontsize = 12,rotation=0)
ax.set_yticklabels(ax.get_ymajorticklabels(), fontsize = 12,rotation=0)

plt.savefig(confusion_matrix_save_path+model_name+'.png',dpi=500)
plt.show()

ax = heatmap(cm_norm, cmap='Blues', annot=True,linecolor='lightblue',linewidths=.5,annot_kws={'size': 14,"weight": "bold"}, xticklabels=['COVID19', 'NORMAL','PNEUMONIA'], yticklabels=['COVID19', 'NORMAL','PNEUMONIA'], square=True, fmt='.2f')

ax.set_xticklabels(ax.get_xmajorticklabels(), fontsize = 12)
ax.set_yticklabels(ax.get_ymajorticklabels(), fontsize = 12,rotation=0)

plt.savefig(confusion_matrix_save_path+model_name+'_normalized.png',dpi=500)